In [1]:
!pip install --upgrade pip

In [2]:
!pip install --upgrade jax jaxlib
!pip install --upgrade flax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.3 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 MB 429.5 kB/s eta 0:00:0000:0100:04
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.23
    Uninstalling jaxlib-0.4.23:
      Successfully uninstalled jaxlib-0.4.23
  Attempting uninstall: jax
    Found existing installation: jax 0.4.23
    Uninstalling jax-0.4.23:
      Successfully uninstalled jax-0.4.23
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.6/677.6 kB 1.8 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: flax
    Found existing installation: flax 0.8.0
    Uninstalling flax-0.8.0:
      Successfully uninstalled flax-0.8.0


# Mamba Architecture
<img src="assets/mamba_architecture.png"/>

## SSM Representaion 3
<img src="assets/ssm.png"/>

#### Continuous

\begin{align}
x`(t) &= A x(t) + B u(t) \\
y(t) &= C x(t) + D u(t)
\end{align}

#### Recurrent

\begin{align}
x(t) &= \overline{A} x_{(t-1)} + \overline{B} u_{(t)} \\
y(t) &= C x(t) + D u(t)
\end{align}

### Convolutional

\begin{align}
y(t) &= \overline{K} * u(t)
\end{align}

## Lets Implement SSM here